In [ ]:
# Import relevant libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import xlsxwriter
import time

In [ ]:
firefox_profile = webdriver.FirefoxProfile()
# Disable images
firefox_profile.set_preference('permissions.default.image', 2)
driver = webdriver.Firefox(firefox_profile=firefox_profile)

In [ ]:
index_pages = ['https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/a-c',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/d-f',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/g-i',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/j-l',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/m-o',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/p-r',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/s-u',
              'https://www.e-fellows.net/Studium/Stipendien/Stipendien-Datenbank/Uebersicht-aller-Stipendien/(range)/v-z']
all_scholarships = []
for i in range(len(index_pages)):
    # Load account page
    driver.get(index_pages[i])
    scholar_link_path = '//ul/li/div/div/h3/a'
    link_elems = driver.find_elements(by=By.XPATH, value=scholar_link_path)
    links = [e.get_attribute('href') for e in link_elems]
    for li in links:
        all_scholarships.append(li)
    

In [ ]:
def scholar_page():
    time.sleep(1)
    head_path = '//article/div[@class="article__head"]'
    head_elems = driver.find_elements(by=By.XPATH, value=head_path)
    heading = []
    for p in range(len(head_elems)):
        heading.append(head_elems[p].text)
    body_path = '//article/div[@class="article__body"]'
    body_elems = driver.find_elements(by=By.XPATH, value=body_path)
    body = []
    for p in range(len(body_elems)):
        body.append(body_elems[p].text)
    return body

In [ ]:
def information(body):
    body = body[0].split('\n')
    scholarship = body[1]
    scholarship = scholarship.split(":")
    Foundation = scholarship[0]
    if len(scholarship) == 2:
        name_of_scholarship = scholarship[1]
    else:
        name_of_scholarship = ''

    try:
        funded_disciplines_index = body.index('Geförderte Fachrichtungen')
    except:
        funded_disciplines_index = -1
    try:
        education_phase_index = body.index('Bildungsphase')
        education_phase_body = body[education_phase_index + 1 : funded_disciplines_index]
        education_phase_body = '\n'.join(education_phase_body)
    except:
        education_phase_index = funded_disciplines_index
        education_phase_body = ''
    try:
        target_regions_index = body.index('Zielregionen')
        target_regions_body = body[target_regions_index + 1 : education_phase_index]
        target_regions_body = '\n'.join(target_regions_body)
    except:
        target_regions_index = education_phase_index
        target_regions_body = ''
    try:
        type_of_schol_index = body.index('Art des Stipendiums')
        type_of_scholarship_body = body[type_of_schol_index + 1: target_regions_index]
        type_of_scholarship_body = '\n'.join(type_of_scholarship_body)
    except:
        type_of_schol_index = target_regions_index
        type_of_scholarship_body = ''
    try:
        requirements_index = body.index('Voraussetzungen')
        requirements_body = body[requirements_index + 1: type_of_schol_index]
        requirements_body = '\n'.join(requirements_body)
    except:
        requirements_index = type_of_schol_index
        requirements_body = ''
    try:
        promotion_index = body.index('Art der Förderung')
        promotion_body = body[promotion_index + 1 : requirements_index]
        promotion_body = '\n'.join(promotion_body)
    except:
        promotion_index = requirements_index
        promotion_body = ''
    try:
        general_purpose_index = body.index('Allgemeines Anliegen des Stipendiums')
        general_body = body[general_purpose_index + 1 : promotion_index]
        general_body = '\n'.join(general_body)
    except:
        general_purpose_index = promotion_index
        general_body = ''

    all_data = [Foundation, name_of_scholarship, general_body, promotion_body, requirements_body,
               type_of_scholarship_body, target_regions_body, education_phase_body]
    return all_data

In [ ]:
workbook = xlsxwriter.Workbook('e-fellow.xlsx')
worksheet = workbook.add_worksheet("sheet_1")
# Writing column names
worksheet.write(0,0, 'Foundation')
worksheet.write(0,1, 'Name of Scholarship')
worksheet.write(0,2, 'General Purpose of Scholarship')
worksheet.write(0,3, 'Type of Promotion')
worksheet.write(0,4, 'Requirements')
worksheet.write(0,5, 'Type of Scholarship')
worksheet.write(0,6, 'Target Regions')
worksheet.write(0,7, 'Education phase')

In [ ]:
row = 1
col = 0

for page in all_scholarships:
    try:
        driver.get(page)
        body = scholar_page()
        all_data = information(body)
        worksheet.write(row, col, all_data[0])
        worksheet.write(row, col+1, all_data[1])
        worksheet.write(row, col+2, all_data[2])
        worksheet.write(row, col+3, all_data[3])
        worksheet.write(row, col+4, all_data[4])
        worksheet.write(row, col+5, all_data[5])
        worksheet.write(row, col+6, all_data[6])
        worksheet.write(row, col+7, all_data[7])
    
        print(row)
        row += 1
        col = 0
    except Exception as e:
        workbook.close()
        print(e)
        break
workbook.close()